In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import mne
# from mne.minimum_norm import read_inverse_operator, source_induced_power
from mne.time_frequency import tfr_morlet

ModuleNotFoundError: No module named 'mne'

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp
from tbd_eeg.tbd_eeg.data_analysis.Utilities.utilities import find_nearest

In [4]:
%matplotlib qt

### Functions

In [5]:
# need to make a function that chooses stim event times
def get_stim_events(stim_table, stim_type, stim_param, sweep, remove_bad_epochs=True):

    if 'good' in stim_table.columns and remove_bad_epochs:
        return stim_table[
                (stim_table['stim_type'] == stim_type) &
                (stim_table['parameter'] == stim_param) &
                (stim_table['sweep'] == sweep) &
                (stim_table['good'] == True)
                ].onset.values
    else:
        return stim_table[
                (stim_table['stim_type'] == stim_type) &
                (stim_table['parameter'] == stim_param) &
                (stim_table['sweep'] == sweep)
                ].onset.values

In [6]:
# use get_stim_events to make mne events array
def get_mne_events(eeg_timestamps, stim_table, stim_type, event_dict, stim_param=None, stim_sweep=None):
    '''
    This function finds the master times for specified events, then extracts the index of the EEG sample associated with them.
    It returns an array ((n_events, 3), compatible with the mne package) with the following info: [[EEG sample of event, value
    of previous event (not relevant), sweep number (can be tied to event_id)],...].
    '''

    mne_events = np.empty([0, 3], dtype=int)
    for event_name, event_param in event_dict.items():
        if stim_sweep is None:
            etimes = get_stim_events(stim_table, stim_type, stim_param, event_param)
        elif stim_param is None:
            etimes = get_stim_events(stim_table, stim_type, event_param, stim_sweep)
        eeg_sample_inds = np.array([find_nearest(eeg_timestamps, x) for x in etimes]) # sample inds for first column
        blank_second_column = np.zeros_like(eeg_sample_inds) # zeros for second column
        eventid_column = np.full(eeg_sample_inds.shape, event_param) # event id for third column
        # column stack and concatenate with all sweep events
        mne_events = np.concatenate((mne_events, np.column_stack((eeg_sample_inds, blank_second_column, eventid_column))), axis=0)
    
    return mne_events[mne_events[:,0].argsort()]

### Get meta data from files

In [7]:
# data_folder = r"E:\EEG_exp\mouse521885\estim1_2020-07-09_14-23-49\recording1"
data_folder = r'F:\EEG_exp\mouse569064\estim_vis_2021-04-08_10-28-24\experiment1\recording1'
exp = EEGexp(data_folder, preprocess=False)

Experiment type: electrical and sensory stimulation
SomnoSuite log file not found.


In [8]:
print(exp.mouse)
print(exp.date)
print(exp.experiment_type)
print(exp.computerID)
print(exp.ephys_params['EEG']['sample_rate'])

569064
2021-04-08 10:28:24
electrical and sensory stimulation
W10DT05491
2500


### Use EEGexp function to load data. Array returned is already remapped into EEG array ch numbers (ch 31 & 32 should be very noisy). Data is in microvolts.

In [9]:
eegdata, eegtimestamps = exp.load_eegdata()

In [10]:
print(eegdata.shape)

(25378816, 30)


In [11]:
print(eegtimestamps[0:10])
print(eegtimestamps[0]*exp.ephys_params['EEG']['sample_rate'])
print(round(eegtimestamps[0]*exp.ephys_params['EEG']['sample_rate'])/exp.ephys_params['EEG']['sample_rate'])

[35.44151405 35.44191405 35.44231405 35.44271405 35.44311405 35.44351405
 35.44391405 35.44431405 35.44471405 35.44511405]
88603.78511696354
35.4416


### Create MNE Raw object

In [11]:
ch_list = np.arange(0, 30)
ch_names = [str(x) for x in ch_list]

info = mne.create_info(ch_names=ch_names, sfreq=exp.ephys_params['EEG']['sample_rate'], ch_types='eeg')
# can add bad channels to info

In [12]:
mne_raw = mne.io.RawArray(eegdata.T[ch_list,:]*1e-6, info)
# for mne.io.RawArray, eeg channels must be in Volts

Creating RawArray with float64 data, n_channels=30, n_times=25378816
    Range : 0 ... 25378815 =      0.000 ... 10151.526 secs
Ready.


### The following cell should create a pop-up plot that allows you to scroll through the whole recording

In [ ]:
scalings = {'eeg': 100e-6}
mne_raw.plot(n_channels=10, scalings=scalings)

Plotting appears to be working. Should double check the scale bar at some point, but looks close enough.

## The rest of this notebook allows for plotting epochs of time

### Create "events" array to make Epochs

From mne docs (https://mne.tools/stable/auto_tutorials/simulation/plot_creating_data_structures.html#tut-creating-data-structures):
<br>*It is necessary to supply an “events” array in order to create an Epochs object. This is of shape(n_events, 3) where the first column is the sample number (time) of the event, the second column indicates the value from which the transition is made from (only used when the new value is bigger than the old one), and the third column is the new event value.*

In [14]:
stim_log = pd.read_csv(exp.stimulus_log_file)
stim_log.head()

,stim_type,parameter,onset,offset,duration,sweep,good,mean_speed,resting_trial
0,biphasic,40,375.08634,375.08674,0.0004,0,True,0.0,True
1,biphasic,15,378.61936,378.61976,0.0004,0,True,0.0,True
2,biphasic,40,382.68098,382.68138,0.0004,0,True,0.0,True
3,biphasic,60,387.06326,387.06366,0.0004,0,True,0.0,True
4,biphasic,60,391.23287,391.23327,0.0004,0,True,0.0,True


In [ ]:
print(stim_log.sweep.unique())

In [ ]:
print(stim_log.parameter.unique())

In [15]:
# # make event_id for SWEEPS to get event array
# event_id_dict = dict(awake=0, anesthetized=1)
# print(event_id_dict)
# ch_events = get_mne_events(eegtimestamps, stim_log, stim_type='biphasic', event_dict=event_id_dict, stim_param=50)
# print(ch_events.shape)
# # might be able to automate this in the future...

# make event_id for AMPLITUDES to get event array
event_id_dict = {}
for ampi in stim_log.parameter.unique():
    event_id_dict[str(ampi)] = ampi
print(event_id_dict)
ch_events = get_mne_events(eegtimestamps, stim_log, stim_type='biphasic', event_dict=event_id_dict, stim_sweep=0)
print(ch_events.shape)

{'40': '40', '15': '15', '60': '60', 'white': 'white'}
(360, 3)


### Use events to extract epochs

In [16]:
ch_epochs = mne.Epochs(mne_raw, ch_events, tmin=-0.5, tmax=1.0, event_id=event_id_dict)

TypeError: events should be a NumPy array of integers, got <class 'numpy.ndarray'>

In [ ]:
ch_epochs.plot(scalings=scalings, n_epochs=5, n_channels=10, events=ch_events, event_id=event_id_dict)

Use Epochs to plot topomap

In [17]:
awake_epochs = epochs_50['awake'].get_data()
anest_epochs = epochs_50['anesthetized'].get_data()

NameError: name 'epochs_50' is not defined

## Pick electrode to analyze

In [ ]:
# plot some single trials stacked
plot_ch = 4
plot_trials = [0, 5, 10, 15, 20, 25, 30, 45, 55, 59]
offset = 0.0005

In [ ]:
plotsavefolder = os.path.join(plotsavedir, 'ch'+str(plot_ch))

if not os.path.exists(plotsavefolder):
    os.mkdir(plotsavefolder)
    print(plotsavefolder)

In [ ]:
fig, ax = plt.subplots(figsize=(5.8, 5.2))
for indi, trial_num in enumerate(plot_trials):
    ax.plot(epochs_50.times, awake_epochs[trial_num, plot_ch, :]+offset*indi, color='k', linewidth=0.8)
ax.set_xlim([epochs_50.times[0], epochs_50.times[-1]])
ax.axvline(0, c='g', linewidth=1, alpha=0.8)
ax.set_title('Awake, 50 uA stim\nCh %d' % plot_ch)
ax.set_xlabel('Time from stim (s)')
ax.tick_params(labelleft=False)

# save as single_trials_awake (inside ch'plot_ch' folder)
plt.savefig(os.path.join(plotsavefolder, 'single_trials_awake.png'), format='png')

In [ ]:
fig, ax = plt.subplots(figsize=(5.8, 5.2))
for indi, trial_num in enumerate(plot_trials):
    ax.plot(epochs_50.times, anest_epochs[trial_num, plot_ch, :]+offset*indi, color='k', linewidth=0.8)
ax.set_xlim([epochs_50.times[0], epochs_50.times[-1]])
ax.axvline(0, c='g', linewidth=1, alpha=0.8)
ax.set_title('Anesthetized, 50 uA stim\nCh %d' % plot_ch)
ax.set_xlabel('Time from stim (s)')
ax.tick_params(labelleft=False)

# save as single_trials_anest
plt.savefig(os.path.join(plotsavefolder, 'single_trials_anest.png'), format='png')